# Color based NDT processing demonstration (trial run)

In [1]:
# Using 2D projection map generated from <color_NDT_preprocessing.ipynb> 
# Attempting initial implementation of color based NDT

Checklist 
1. [x] 2D projection map creation
2. [x] Satellite image scaling
3. [ ] Common coordinte frame with initial guess
4. [ ] Define bins along coordinate frame
5. [ ] Distribute reference map points in corresponding bins
6. [ ] Distribute satellite image points in corresponding bins
7. [ ] Define distributions within each voxel
8. [ ] Define gradient data within each voxel 

Other things to do:
1. [ ] Come up with a better acronym than color based NDT (if it works)
2. [ ] Revisit scaling methods

In [2]:
import numpy as np
import cv2
import open3d as o3d
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import imageio

# If necessary 
from groundNAV_utils import * 
from colmapParsingUtils import *

# SAVE YOUR WORK
%load_ext autoreload
%autoreload 2
%autosave 180

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Autosaving every 180 seconds


In [3]:
# Load in reference map and satellite image 
ref_scene = np.load('Data_arrays/turf_colmap_2d_proj.npy')
ref_rgb = np.load('Data_arrays/turf_colmap_2d_rgb.npy')
sat_im = cv2.imread('TTurf/TurfSat.jpg')
sat_im_crop = cv2.imread('TTurf/TurfSat_crop.png')

In [4]:
# # Optionally, display the resized image
# # Can do with opencv, don't need open3d
# cv2.imshow('Satellite image', sat_im)
# cv2.imshow('Satellite image cropped', sat_im_crop)

# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [5]:
# Create 2D points from image (1 in z direction)
# print(sat_im_crop)
sat_img = cv2.cvtColor(sat_im_crop, cv2.COLOR_BGR2RGB)
L = sat_img.shape[0]
W = sat_img.shape[1]
n = L*W
sat_pts = np.zeros((n,3))
sat_rgb = np.zeros((n,3))
# print(sat_pts)
count = 0
for i in range(L):
    for j in range(W):
        px = j
        py = j
        rgb = sat_img[i][j]
        sat_pts[count] = [i, j, 1]
        sat_rgb[count] = rgb
        count += 1

focal = 2987.396087478296
print("\n Focal length = ", focal)
ref_scene[:, :2] *= focal
sat_rgb = sat_rgb/255
print("Satellite points\n", sat_pts)
print("\nRGB values \n", sat_rgb)


 Focal length =  2987.396087478296
Satellite points
 [[  0.   0.   1.]
 [  0.   1.   1.]
 [  0.   2.   1.]
 ...
 [499. 247.   1.]
 [499. 248.   1.]
 [499. 249.   1.]]

RGB values 
 [[0.37647059 0.45490196 0.24705882]
 [0.36862745 0.44705882 0.23921569]
 [0.36862745 0.44705882 0.23921569]
 ...
 [0.40784314 0.4745098  0.2627451 ]
 [0.41568627 0.49019608 0.2745098 ]
 [0.41176471 0.49411765 0.2745098 ]]


In [6]:
# Use open3d to create point cloud visualization 

# Create visualization 
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="Originial scene with origin")

# Create axes @ origin
axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100)

# Create point cloud for reference cloud
ref_cloud = o3d.geometry.PointCloud()
ref_cloud.points = o3d.utility.Vector3dVector(ref_scene)
ref_cloud.colors = o3d.utility.Vector3dVector(ref_rgb)

# Create point cloud for satellite image
sat_cloud = o3d.geometry.PointCloud()
sat_cloud.points = o3d.utility.Vector3dVector(sat_pts)
sat_cloud.colors = o3d.utility.Vector3dVector(sat_rgb)

# Add necessary geometries to visualization 
vis.add_geometry(ref_cloud)
vis.add_geometry(sat_cloud)
vis.add_geometry(axis_origin)

# # Size options (jupyter gives issues when running this multiple times, but it looks better)
# render_option = vis.get_render_option()
# render_option.point_size = 2

# Run and destroy visualization 
vis.run()
vis.destroy_window()

libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 69
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 69
pci id for fd 69: 8086:a7a0, driver iris
MESA-LOADER: dlopen(/usr/lib/x86_64-linux-gnu/dri/iris_dri.so)
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using 

In [7]:
# PLOT NEW POINTS 

vis = o3d.visualization.Visualizer()
vis.create_window()

# Add coordinate axes
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100)
# axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)


# cloud_2d = o3d.geometry.PointCloud()
# cloud_2d.points = o3d.utility.Vector3dVector(pts_2d)
# # cloud_2d.paint_uniform_color([0, 0, 1])
# cloud_2d.colors = o3d.utility.Vector3dVector(camera_pts_rgb)

# Add necessary geometries
# vis.add_geometry(axis_origin)
vis.add_geometry(ref_cloud)
vis.add_geometry(sat_cloud)


render_option = vis.get_render_option()
render_option.point_size = 2

vis.poll_events()
vis.update_renderer()

# Set up initial viewpoint
view_control = vis.get_view_control()
# Direction which the camera is looking
view_control.set_front([0, 0, -1])  # Set the camera facing direction
# Point which the camera revolves about 
view_control.set_lookat([0, 0, 0])   # Set the focus point
# Defines which way is up in the camera perspective 
view_control.set_up([-1, 0, 0])       # Set the up direction
view_control.set_zoom(2)           # Adjust zoom if necessary


# Capture frames for GIF
frames = []
num_frames = 30  # Adjust the number of frames
angle_step = 180/num_frames


for i in range(num_frames):
	# Rotate the view
    view_control.rotate(angle_step, 0)  # (horizontal, vertical)

    # vis.update_geometry(axis_orig) # Only if I move it myself?
    vis.poll_events()
    vis.update_renderer()

    # Capture frame directly into memory
    image = vis.capture_screen_float_buffer(False)
    image_8bit = (np.asarray(image) * 255).astype(np.uint8)  # Convert to 8-bit
    frames.append(image_8bit)

for i in range(num_frames):
	# Rotate the view
	view_control.rotate(-angle_step, 0)  # (horizontal, vertical)

	# vis.update_geometry(axis_orig) # Only if I move it myself?
	vis.poll_events()
	vis.update_renderer()

	# Capture frame directly into memory
	image = vis.capture_screen_float_buffer(False)
	image_8bit = (np.asarray(image) * 255).astype(np.uint8)  # Convert to 8-bit
	frames.append(image_8bit)

for i in range(num_frames):
	# Rotate the view
	view_control.rotate(-angle_step, 0)  # (horizontal, vertical)

	# vis.update_geometry(axis_orig) # Only if I move it myself?
	vis.poll_events()
	vis.update_renderer()

	# Capture frame directly into memory
	image = vis.capture_screen_float_buffer(False)
	image_8bit = (np.asarray(image) * 255).astype(np.uint8)  # Convert to 8-bit
	frames.append(image_8bit)
    
for i in range(num_frames):
	# Rotate the view
	view_control.rotate(angle_step, 0)  # (horizontal, vertical)

	# vis.update_geometry(axis_orig) # Only if I move it myself?
	vis.poll_events()
	vis.update_renderer()

	# Capture frame directly into memory
	image = vis.capture_screen_float_buffer(False)
	image_8bit = (np.asarray(image) * 255).astype(np.uint8)  # Convert to 8-bit
	frames.append(image_8bit)



# Create GIF
# Ensure frames are in the correct format
frames = [frame.astype("uint8") for frame in frames]

# Use imageio to save as GIF
imageio.mimsave("cNDT_initial.gif", frames, fps=30, loop=0)  # Adjust fps if necessary

# Run visualization 
vis.run()
vis.destroy_window()

libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 69
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 69
pci id for fd 69: 8086:a7a0, driver iris
MESA-LOADER: dlopen(/usr/lib/x86_64-linux-gnu/dri/iris_dri.so)
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using 

# Plotting Tools

In [ ]:
# Use open3d to create point cloud visualization 

# Create visualization 
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="Originial scene with origin")

# Create axes @ origin
axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)

# Create point cloud for scene pts
scene_cloud = o3d.geometry.PointCloud()
scene_cloud.points = o3d.utility.Vector3dVector(ref_scene)
scene_cloud.colors = o3d.utility.Vector3dVector(ref_rgb)

# Add necessary geometries to visualization 
vis.add_geometry(axis_origin)
vis.add_geometry(scene_cloud)

# Size options (jupyter gives issues when running this multiple times, but it looks better)
render_option = vis.get_render_option()
render_option.point_size = 2

# Run and destroy visualization 
vis.run()
vis.destroy_window()